<a href="https://colab.research.google.com/github/otanet/Quantum_Computing_Annealing_Machine_20211209/blob/main/noise_reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 画像のノイズリダクション

## 問題の概要

アニーリングマシンを用いてノイズが加わった画像から元の画像の推定を行うことを考えます。

下記の仮定に基づいてノイズ除去を試みます。

*   元の画像とノイズの入った画像はほとんど一致することが多い
*   元の画像では隣り合う画素は同じ色であることが多い

ここでは簡単のため白黒画像を取り扱います。  
画素のデータは黒と白の二値での表現が可能なので、各画素の値を二値変数を用いて表すことができます。  
上記の仮定を表現する画素同士の相互作用を目的関数を用いて表現し、これを最適化することによって元の画像を推定することができます。

## 目的関数の構築

画素の集合を $V$ とし、各画素を表すインデックスを $i\in V$ とします。  
まず、入力画素を表すイジング変数を $y$ としそれぞれの色に対応する値を以下のように表します。

$$
y_{i} = \left\{
\begin{align}
&+1 \quad\text{(白)}\\
&-1 \quad \text{(黒)}
\end{align}
\right. \quad
i\in V\\
$$

また、出力画素に対応した二値のイジング変数を以下のように表します。

$$
s_{i} = \left\{
\begin{align}
&+1 \quad\text{(白)}\\
&-1 \quad \text{(黒)}
\end{align}
\right. \quad
i\in V\\
$$

入力画像と出力画像は概ね一致するという仮定 (ノイズがそれほど多くないという仮定) により、入力画素と出力画素は同じ値になるようにします。つまり、$s_i$ と $y_i$ は同じ値を持つときに値が小さくなるような目的関数を関数を導入します。例えば以下のように与えられます。

$$
f_1 = - \sum_{i\in V} y_{i} s_{i}
$$

$y_{i}$ と $s_{i}$ が同じ値を持つと 上記の目的関数の値は減少し、異なった値を持つと増加するので、全ての $i\in V$ において $y_{i} = s_{i}$ である場合に $f_1$ は最小値をとります。しかしながら、入力画像にはノイズがのっているので、出力画像が入力画像と同じになってしまうとノイズを減らすことができません。

そこで、隣り合う画素は同じ色になりやすいという仮定を考慮します。  
隣り合う出力画素がなるべく同じ値を持つような関数を導入し、ノイズと思われる画素に対しその周りの画素の値と一致させることでノイズを減らすことを考えます。例えば以下のような関数が考えられます。
 
$$
f_2 = -\sum_{(i,j)\in E} s_i s_j
$$

ここで隣接する画素のペアの集合を $E$ としました。隣り合った出力画素が同じ値を持つとこの関数の値は減少します。したがって、全ての出力画素が同じ値を持つと $f_2$ は最小の値をとることがわかります。しかし、全ての画素が同じ値になってしまうと全てが白または黒の画像になってしまうので、元の画像の情報が失われてしまいます。

そこで、$f_1$ と $f_2$ を適切に足し合わせることで、出力画像が入力画像と近い値をとりつつノイズと思われる画素のみ除去することを試みます。

$$
\begin{align}
f & = f_1 + \eta f_2\\
&=- \sum_{i\in V}y_is_i - \eta \sum_{(i,j)\in E}s_i s_j
\end{align}
$$

ここで、$\eta>0$ というパラメータを導入しました。これにより $f_1$ と $f_2$ の強さの調整が出来ます。$\eta$ が大きいほどノイズ除去を行う項が強いことを意味しています。

この目的関数を最小化しイジング変数 $s$ の値を画素の値と解釈することで、ノイズを除去した画像が得られます。


## 参考
* [Annealing Cloud Web: デモアプリ](https://annealing-cloud.com/ja/play/demoapp/noise.html)
* [Annealing Cloud Web: 画像のノイズリダクション解説](https://annealing-cloud.com/ja/tutorial/2.html)

## 必要なライブラリのインストール

In [ ]:
! pip install -q amplify
! pip install -q matplotlib
! pip install -q Pillow

## 画像の読み込み

まずは、画像データをダウンロードする関数と、ダウンロードした画像をイジング変数配列に変換する関数を定義します。

In [ ]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import urllib.error
import urllib.request

# 画像をダウンロードして保存
def download_image_file(url, dst_path):
    try:
        with urllib.request.urlopen(url) as web_file:
            data = web_file.read()
            with open(dst_path, mode="wb") as local_file:
                local_file.write(data)
    except urllib.error.URLError as e:
        print(e)


def get_ising_img_array(path, max_size=196, threshold=128):
    # 画像を読み込む
    img = Image.open(path)

    # 画像をリサイズ
    width, height = img.size
    if width > max_size or height > max_size:
        img.thumbnail((max_size, max_size), Image.ANTIALIAS)

    # グレースケールに変換しイジング変数配列に格納
    # 二値に分ける閾値は threshold 引数で変更可能
    return np.where(np.array(img.convert("L")) >= threshold, 1, -1)

画像をダウンロードします。  
以下の例ではレナ画像を Wikipedia からダウンロードします。  
任意の画像のURLに変更も可能です。

In [ ]:
url = "https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_(test_image).png"
dst_path = "web_fig.png"  # 画像の保存先
download_image_file(url, dst_path)

保存した画像をイジング変数配列に変換します。

In [ ]:
# 元画像のイジング配列を作成
x = get_ising_img_array(dst_path)
plt.imshow(x, cmap="gray")
plt.show()
print(x.shape)
print(x)

## ノイズ画像の作成

次に、画素を無作為に選びその値を反転することでノイズを表現する関数を定義します。

In [ ]:
def get_noisy_img_array(img_array):
    # 2次元配列を1次元に変換して扱いやすくする
    img_shape = img_array.shape
    flattened_img = img_array.flatten()

    # 最大値と最小値を入れ替える関数を定義
    min_v = min(flattened_img)
    max_v = max(flattened_img)

    def invert_value(v):
        return min_v + max_v - v

    # ノイズの割合
    ratio = 0.02

    # ノイズをのせる画素をランダムに選択して反転
    for idx in np.random.choice(len(flattened_img), int(ratio * len(flattened_img))):
        flattened_img[idx] = invert_value(flattened_img[idx])

    # 元の配列の形に戻す
    return flattened_img.reshape(*img_shape)

In [ ]:
# ノイズ画像のイジング配列を作成
y = get_noisy_img_array(x)
plt.imshow(y, cmap="gray")
plt.show()
print(y.shape)
print(y)

## イジング変数配列の作成

次に、イジング変数の配列 `s` を生成します。入力画像のデータ `y` を $h \times w$ の2次元配列とすると、出力画像に対応するイジング変数 `s` も 同じく $h \times w$ の2次元配列となります。

変数の生成には [`SymbolGenerator`](https://amplify.fixstars.com/ja/docs/polynomial.html?#polynomial-symbol-generator) を使います。ここでは、最終的にイジング変数の多項式が目的関数となるので、変数の種類を [`IsingPoly`](https://amplify.fixstars.com/ja/docs/polynomial.html?#id1) に指定します。[`SymbolGenerator`](https://amplify.fixstars.com/ja/docs/polynomial.html?#polynomial-symbol-generator) では、入力画像のデータ `y` と同じ配列の形で作成することが可能なので、この機能を使うと目的関数の計算に便利です。

In [ ]:
from amplify import IsingSymbolGenerator, IsingPoly, einsum

# h x w の配列の形にイジング変数を生成
gen = IsingSymbolGenerator()
s = gen.array(y.shape)

## 目的関数

入力画像データの配列 $y$ と出力画像に対応したイジング変数配列 $s$ を用いて、目的関数を構築します。

### 課題1

下記のコメント部分を実装して目的関数を計算して下さい。和の記号は行方向`h`と列方向`w`の両方に必要な事に注意して下さい。

ヒント: [多項式の演算子](https://amplify.fixstars.com/ja/docs/polynomial.html?#id3) を参考に構築してみましょう。

In [ ]:
# 画像のシェイプを取得
h, w = y.shape

# 強度パラメータ
eta = 0.333

# 目的関数を f を計算
# (f1とf2を下記のコメントに沿って生成)

# - \sum_{i\in V} y_{i} s_{i}
# f1 = IsingPoly()
# for i in range(h):
#     for j in range(w):
#         f1 -= s[i, j] * ...

# -\sum_{(i,j)\in E} s_i s_j
# f2 = IsingPoly()
# for i in range(h):
#     for j in range(w):
#         f2 -= s[i, j] * ...
f = f1 + eta * f2

また、`sum` メソッドや [`einsum` 関数](https://amplify.fixstars.com/ja/docs/polynomial.html?#polynomial-einsum) を用いることによって、以下のように記述することもできます。

In [ ]:
# f1 = -(y * s).sum()
# もしくは
# f1 = -einsum("ij,ij->", y, s)

# s_col_roll = s.roll(-1, 1)
# for i in range(s_col_roll.shape[0]):
#     s_col_roll[i, -1] = IsingPoly(0)
#
# s_row_roll = s.roll(-1, 0)
# for i in range(s_row_roll.shape[1]):
#     s_row_roll[-1, i] = IsingPoly(0)

# f2 = - (s * s_col_roll).sum() - (s * s_row_roll).sum()
# もしくは
# f2 = - einsum("ij,ij->", s, s_col_roll) - einsum("ij,ij->", s, s_row_roll)

## クライアントの設定とマシンの実行

次にクライアントを設定し、先ほど与えたコスト関数の最小値に対応する解をイジングマシンで探索します。

In [ ]:
from amplify.client import FixstarsClient
from amplify import Solver

# クライアントの設定
client = FixstarsClient()
client.parameters.timeout = 1000  # タイムアウト1秒
client.token = "トークンを入力してください"

# ソルバの設定と結果の取得
solver = Solver(client)
result = solver.solve(f)

## 解の取得と結果の表示

最後に、得られた解を元のイジング変数 $s$ に代入し、出力画像のデータを取得します。

入力画像と比較するとノイズが減少したことが確認できます。

In [ ]:
# 解の取得
values = result.solutions[0].values

# イジング変数に解を代入
output = s.decode(values, 1)

plt.imshow(output, cmap="gray")  # 復元画像
plt.show()

plt.imshow(x, cmap="gray")  # 元画像
plt.show()

plt.imshow(y, cmap="gray")  # ノイズ画像
plt.show()

## 発展課題

### 課題2

ダウンロードする画像を変更して正常に実行されるか確認して下さい

### 課題3

ノイズ量やノイズ除去の強さを変更して結果がどのように変化するかを確認して下さい